In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import re, os, csv, pathlib
import pandas as pd
from statistics import mean, variance
accents = ['african', 'indian', 'hongkong', 'philippines', 
           'england', 'scotland', 'ireland', 'australia', 
           'canada', 'us', 
           'bermuda', 'southatlandtic', 'wales', 'malaysia']
accent_short_forms = {'hongkong':'hk', "african":"afr", "philippines":"phil", "indian":"ind",  
                      "england":"eng", 'scotland':'sco', 'ireland':'ire',
                      "us":"us", "canada":"can", "australia":"aus",
                      'bermuda':'ber', 'southatlandtic':'satl', 'wales':'wal', 'malaysia':'mal'}

def replace_with_short_forms(s):
    for key, value in accent_short_forms.items():
        s = s.replace(key, value)
    return s

def last_name(pth):
    return pathlib.PurePath(pth).name

def get_dirs(pth):
    return [last_name(f.name) for f in os.scandir(pth) if f.is_dir()]

def get_each_run(lne):
    return list(map(float, re.findall(': (.+?) -> ', lne)[0].split(' ')))

def get_selection_counts(s):
    return list(map(replace_with_short_forms, re.findall('Counter\\((.+?)\\)', s)))

def get_test_file_from_stats_path(run_number, stats_file_opened):
    return stats_file_opened.name[:-9]+"run_{}/output/test_infer_log.txt".format(run_number)

def WER_test_file(test_file):
    txt_file = open(test_file, 'r')
    lines = txt_file.readlines()
    matched = ""
    for line in lines:
        if "==========>>>>>>Evaluation Greedy WER: " in line:
            txt_file.close()
            return float(line.rstrip().split(": ")[1])
    txt_file.close()
    return ""

def get_eta(func, eta):
    if func != "FL1MI":
        return ""
    else:
        return "-n:"+str(float(eta[4:]))

In [2]:
budget = 200
target = 20
features = 'wv10_100'
# features = '39'
# sample_path = 'Error-Driven-ASR-Personalization/CMU_expts/accent/hindi/manifests/TSS_output/all/budget_100/target_50/FL1MI/eta_1.0/euclidean/39/stats.txt'
cols = ['accent', 'ground', 'function', 'feature', 'duration', 'samples', 
        'WER-r1', 'WER-r2', 'WER-r3', 'WER-mean', 'WER-stdev', 'accents-run_1', 'accents-run_2', 'accents-run_3']
df = pd.DataFrame(columns = cols)
# accents = ['canada', 'england', 'australia', 'us', 'indian', 'philippines', 'african']
accents = ['african', 'indian', 'hongkong', 'philippines', 
           'england', 'scotland', 'ireland', 'australia', 
           'canada', 'us', 
           'bermuda', 'southatlandtic', 'wales', 'malaysia']
cnt = 0
pretrain_added = dict([(accent, False) for accent in accents])
for accent in accents:
#     if accent not in ['indian', 'hongkong']: continue
    if not(pathlib.Path('./{}/manifests/TSS_output/'.format(accent)).is_dir()):
        continue
    pretrain_log = '{}/manifests/TSS_output/test_infer_log.txt'.format(accent)
    pick_from = 'all'
    if not(pathlib.Path('./{}/manifests/TSS_output/{}/budget_{}/target_{}/'.format(accent, pick_from, budget, target)).is_dir()):
        continue
    for function in get_dirs('./{}/manifests/TSS_output/{}/budget_{}/target_{}/'.format(accent, pick_from, budget, target)):
        if function == "random":
            stats_file_path = './{}/manifests/TSS_output/{}/budget_{}/target_{}/{}/stats.txt'.format(accent, pick_from, budget, 
                                                                                                                target, function)                                                                                  
            if not(os.path.isfile(stats_file_path)):
                continue
            stats_file = open(stats_file_path, 'r')
            lines = stats_file.readlines()
            total_selections, total_durations, accented_selections, accented_durations = map(get_each_run, lines[:4])
            sample_frac = mean([x[0]/x[1] for x in zip(accented_selections, total_selections)])
            sample_total = mean(total_selections)
            duration_frac = mean([x[0]/x[1] for x in zip(accented_durations, total_durations)])
            duration_total = mean(total_durations)
            df_duration = "{:.2f}/{:.2f}".format(duration_total*duration_frac, duration_total)
            df_samples = "{:.2f}/{:.2f}".format(sample_total*sample_frac, sample_total)
            df_selections = get_selection_counts(lines[5])
            try:
                wers = [WER_test_file(get_test_file_from_stats_path(i, stats_file)) for i in range(1,4)]
                df_wer_mean = round(mean(wers), 2)
                df_wer_stdev = round(variance(wers), 3)**0.5
            except:
                continue
                print("no WER's in file", get_test_file_from_stats_path(1, stats_file))
                wers = [0,0,0]
                df_wer_mean = 0
                df_wer_stdev = 999
            print(stats_file_path)
            df = df.append(dict(zip(cols, [accent, pick_from, function, "NA", df_duration, df_samples]+wers
                                    +[df_wer_mean, df_wer_stdev] + df_selections)), ignore_index=True)
            stats_file.close()
            continue
        for eta in get_dirs('./{}/manifests/TSS_output/{}/budget_{}/target_{}/{}/'.format(accent, pick_from, budget, target, function)):
            for similarity in get_dirs('./{}/manifests/TSS_output/{}/budget_{}/target_{}/{}/{}/'.format(accent, pick_from, 
                                                                                                    budget, target, function, eta)):
                cnt += 1
                stats_file_path = './{}/manifests/TSS_output/{}/budget_{}/target_{}/{}/{}/{}/{}/stats.txt'.format(accent, pick_from, budget, 
                                                                                                            target, function, eta, similarity, 
                                                                                                            features)                                                                                  
                if not(os.path.isfile(stats_file_path)):
                    continue
                stats_file = open(stats_file_path, 'r')
                lines = stats_file.readlines()
                total_selections, total_durations, accented_selections, accented_durations = map(get_each_run, lines[:4])
                sample_frac = mean([x[0]/x[1] for x in zip(accented_selections, total_selections)])
                sample_total = mean(total_selections)
                duration_frac = mean([x[0]/x[1] for x in zip(accented_durations, total_durations)])
                duration_total = mean(total_durations)
                df_duration = "{:.2f}/{:.2f}".format(duration_total*duration_frac, duration_total)
                df_samples = "{:.2f}/{:.2f}".format(sample_total*sample_frac, sample_total)
                df_selections = get_selection_counts(lines[5])
                try:
                    wers = [WER_test_file(get_test_file_from_stats_path(i, stats_file)) for i in range(1,4)]
                    df_wer_mean = round(mean(wers), 2)
                    df_wer_stdev = round(variance(wers), 3)**0.5
                except:
                    continue
                    print("no WER's in file", get_test_file_from_stats_path(1, stats_file))
                    wers = [0,0,0]
                    df_wer_mean = 0
                    df_wer_stdev = 999
                df = df.append(dict(zip(cols, [accent, pick_from, function+get_eta(function, eta), features, df_duration, df_samples]+wers
                                        +[df_wer_mean, df_wer_stdev] + df_selections)), ignore_index=True)
                if not(pretrain_added[accent]) and os.path.isfile(pretrain_log):
                    pretrain_added[accent] = True
                    pre_WER = WER_test_file(pretrain_log)
                    df = df.append(dict(zip(cols, [accent, "pretrain", '', '', '', '']+['-']*3+[pre_WER, ''] + [])), ignore_index=True)
                stats_file.close()
df = df.sort_values(by=['accent', 'feature', 'ground', 'function'], ascending=True, ignore_index=True)
display(df)

./african/manifests/TSS_output/all/budget_200/target_20/random/stats.txt
./indian/manifests/TSS_output/all/budget_200/target_20/random/stats.txt
./hongkong/manifests/TSS_output/all/budget_200/target_20/random/stats.txt
./philippines/manifests/TSS_output/all/budget_200/target_20/random/stats.txt
./scotland/manifests/TSS_output/all/budget_200/target_20/random/stats.txt
./australia/manifests/TSS_output/all/budget_200/target_20/random/stats.txt
./canada/manifests/TSS_output/all/budget_200/target_20/random/stats.txt
./bermuda/manifests/TSS_output/all/budget_200/target_20/random/stats.txt
./southatlandtic/manifests/TSS_output/all/budget_200/target_20/random/stats.txt
./malaysia/manifests/TSS_output/all/budget_200/target_20/random/stats.txt


,accent,ground,function,feature,duration,samples,WER-r1,WER-r2,WER-r3,WER-mean,WER-stdev,accents-run_1,accents-run_2,accents-run_3
0,african,pretrain,,,,,-,-,-,23.75,,NaN,NaN,NaN
1,african,all,random,NA,42.57/1077.05,7.69/228.00,23.07,23.67,22.93,23.22,0.3937,"{'us': 91, 'eng': 47, 'ind': 27, 'can': 14, 'a...","{'us': 94, 'eng': 37, 'ind': 31, 'aus': 12, 's...","{'us': 94, 'ind': 34, 'eng': 31, 'can': 20, 's..."
2,african,all,FL2MI,wv10_100,960.90/1077.61,178.00/204.00,22.5,22.6,22.54,22.55,0.054772,"{'afr': 178, 'eng': 9, 'us': 7, 'aus': 7, 'ind...","{'afr': 178, 'eng': 9, 'us': 7, 'aus': 7, 'ind...","{'afr': 178, 'eng': 9, 'us': 7, 'aus': 7, 'ind..."
3,african,all,GCMI,wv10_100,948.66/1076.64,175.00/199.00,23.34,22.0,22.43,22.59,0.684105,"{'afr': 175, 'aus': 9, 'us': 8, 'eng': 6, 'ind...","{'afr': 175, 'aus': 9, 'us': 8, 'eng': 6, 'ind...","{'afr': 175, 'aus': 9, 'us': 8, 'eng': 6, 'ind..."
4,african,all,LogDMI,wv10_100,624.74/1077.02,123.00/224.00,22.64,22.23,22.74,22.54,0.270185,"{'afr': 123, 'eng': 36, 'us': 31, 'aus': 15, '...","{'afr': 123, 'eng': 36, 'us': 31, 'aus': 15, '...","{'afr': 123, 'eng': 36, 'us': 31, 'aus': 15, '..."
5,australia,pretrain,,,,,-,-,-,28.76,,NaN,NaN,NaN
6,australia,all,random,NA,57.57/1077.05,12.01/228.00,28.52,27.87,27.68,28.02,0.440454,"{'us': 91, 'eng': 47, 'ind': 27, 'can': 14, 'a...","{'us': 94, 'eng': 37, 'ind': 31, 'aus': 12, 's...","{'us': 94, 'ind': 34, 'eng': 31, 'can': 20, 's..."
7,australia,all,FL2MI,wv10_100,24.30/1078.55,6.00/239.00,28.99,28.99,28.99,28.99,0.0,"{'us': 178, 'eng': 33, 'can': 16, 'aus': 6, 'a...","{'us': 178, 'eng': 33, 'can': 16, 'aus': 6, 'a...","{'us': 178, 'eng': 33, 'can': 16, 'aus': 6, 'a..."
8,australia,all,GCMI,wv10_100,134.80/1077.38,27.00/225.00,26.99,26.75,26.71,26.82,0.151658,"{'us': 107, 'eng': 61, 'aus': 27, 'can': 13, '...","{'us': 107, 'eng': 61, 'aus': 27, 'can': 13, '...","{'us': 107, 'eng': 61, 'aus': 27, 'can': 13, '..."
9,australia,all,LogDMI,wv10_100,121.30/1077.17,25.00/233.00,28.54,28.11,27.57,28.07,0.485798,"{'us': 85, 'eng': 71, 'aus': 25, 'afr': 17, 'i...","{'us': 85, 'eng': 71, 'aus': 25, 'afr': 17, 'i...","{'us': 85, 'eng': 71, 'aus': 25, 'afr': 17, 'i..."


In [59]:
# total selection : 100 100 100 -> 100.00
# total selection duration: 357.0149433106577 357.0149433106577 357.0149433106577 -> 357.01
# accented selection: 76 76 76 -> 76.00
# accented duration: 254.74947845804974 254.74947845804974 254.74947845804974 -> 254.75

# all selections: [Counter({'hindi': 76, 'korean': 8, 'spanish': 7, 'arabic': 3, 'chinese': 3, 'vietnamese': 3}), Counter({'hindi': 76, 'korean': 8, 'spanish': 7, 'arabic': 3, 'chinese': 3, 'vietnamese': 3}), Counter({'hindi': 76, 'korean': 8, 'spanish': 7, 'arabic': 3, 'chinese': 3, 'vietnamese': 3})]

#Evaluation Greedy WER: 16.19

df.to_csv("with_{}_{}_{}.csv".format(budget, target, features), index=False)